In [37]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2

In [38]:
@load "../data/sim3_pars.jld";

In [39]:
# Generate simulation data #
Random.seed!(123);
coords = rand(N, 2);                                          # random location over unit square
X = [fill(1, (N,)) rand(N) rand(N)];                             # design matrix
Dist = pairwise(Euclidean(), coords, dims = 1);                  # distance matrix
F_true = hcat([rand(MvNormal(exp.(-ϕ[i] * Dist)), 1) for i in 1:K]...);
ω = F_true * Λ;                                               # latent process
Y = X * β + ω + [rand(Normal(0.0, sqrt_Σ_diag[i]), 1)[1] for j in 1:N, i in 1:q];              # response matrix
ω_incp = ω + fill(1.0, (N, 1)) * transpose(β[1, :]);

In [40]:
# Some data preparations #

#ordx = sortperm(coords[:, 1]);                                # sort order based on the first coordinates
ordx = 1:N;
X_ord = X[ordx, :]; Y_ord = Y[ordx, :]; ω_ord = ω[ordx, :];   # sorted data
ω_incp_obs = ω_ord + fill(1.0, (N, 1)) * transpose(β[1, :]); # latent process + intercept
coords_ord = coords[ordx, :];
S_ind = [sample(1:N, Nobs, replace = false, ordered = true) for i in 1:q] 
                                                              # index of observed response for all q responses
S = sort(union([S_ind[i] for i in 1:q]...));                  # location with at least one observation
M_ind = [setdiff(S, S_ind[i]) for i in 1:q];                  # in S not in S1
obs_ind = vcat([S_ind[i] .+ (i - 1) * N for i in 1:q]...)     # index of the observed location for all response among N locations
perm_ind = sortperm(vcat([S_ind[i] for i in 1:q]...))

index_S = zeros(N);                                           # build index indicating which response are observed
index_S_M = fill(0.0, N, q);
for i in 1:q
    index_S_M[S_ind[i], i] .= 1;
    index_S = index_S + 2^(i - 1) * index_S_M[:, i]
end
M_Sind = [findall(x -> x == 0.0, index_S_M[S, i]) for i in 1:q];
                                                              # index of M1 among S 

In [41]:
@save "sim3data.jld"